In [ ]:
def _phi_prime(X, Y, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y, Ncores):
    
    import numpy as np
    from tqdm.notebook import tqdm
    from joblib import Parallel, delayed
    
    phi = np.linspace(0, 2*np.pi, 180)
    
    def parallelization(i, X, Y, phi, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y):
        
        phi_prime = np.zeros((X.shape[1], phi.shape[0]))
        
        for j in range(X.shape[1]):
            
            x = np.array([X[i, j], Y[i, j]])

            for p in range(phi.shape[0]):
    
                inner_product_C11 = C11_x[i, j]*np.cos(phi[p])+C11_y[i, j]*np.sin(phi[p])
                inner_product_C12 = C12_x[i, j]*np.cos(phi[p])+C12_y[i, j]*np.sin(phi[p])
                inner_product_C22 = C22_x[i, j]*np.cos(phi[p])+C22_y[i, j]*np.sin(phi[p])

                phi_prime_num = np.cos(phi[p])**2*inner_product_C11
                phi_prime_num += np.sin(2*phi[p])*inner_product_C12
                phi_prime_num += np.sin(phi[p])**2*inner_product_C22
    
                phi_prime_denom = np.sin(2*phi[p])*(C22[i, j]-C11[i, j])
                phi_prime_denom +=  2*np.cos(2*phi[p])*C12[i, j]
    
                phi_prime[j, p] = -phi_prime_num/phi_prime_denom
        
        return phi_prime

    phi_prime = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallelization)(i, X, Y, phi, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y) for i in tqdm(range(X.shape[0]))))

    from scipy.interpolate import RegularGridInterpolator
      
    return RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime)